In [1]:
import sys
import os

sys.path.append(f'{os.getcwd()}/..')

import numpy as np
import pandas as pd
import seaborn as sn
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegressionCV

from collections import Counter

from src.class_computer import link_prediction_methods

In [2]:
df = pd.read_csv('../data/precomputed_with_classes_3.csv')
del df['class']
df = df.dropna()

In [3]:
in_cols = list(df.columns[3:-len(link_prediction_methods)])
out_cols = link_prediction_methods

In [4]:
from keras import Sequential, Input
from keras.layers import Dense

in_dim = len(in_cols)
out_dim = len(out_cols)

def build_nn_model():
    model = Sequential([
        Dense(in_dim*2, input_dim=in_dim, activation='relu'),
        Dense(in_dim*2, activation='relu'),
        Dense(out_dim),
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy', 'mae'])
    return model


Using TensorFlow backend.


In [5]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[in_cols], df[out_cols], test_size=0.10, random_state=0)

normalizer = Normalizer().fit(X_train)

X_train = normalizer.transform(X_train)
Y_train = y_train

X_test = normalizer.transform(X_test)
Y_test = y_test

regressor = KerasRegressor(build_fn=build_nn_model, batch_size=32,epochs=100)
regressor.fit(X_train, Y_train)


Epoch 1/100
1477/1477 [==============================] - 1s 862us/step - loss: 0.4625 - accuracy: 0.2722 - mae: 0.6381
Epoch 2/100
1477/1477 [==============================] - 0s 213us/step - loss: 0.1146 - accuracy: 0.3033 - mae: 0.2782
Epoch 3/100
1477/1477 [==============================] - 0s 169us/step - loss: 0.0191 - accuracy: 0.3182 - mae: 0.1033
Epoch 4/100
1477/1477 [==============================] - 0s 195us/step - loss: 0.0121 - accuracy: 0.3243 - mae: 0.0736
Epoch 5/100
1477/1477 [==============================] - 0s 296us/step - loss: 0.0107 - accuracy: 0.3053 - mae: 0.0671
Epoch 6/100
1477/1477 [==============================] - 0s 309us/step - loss: 0.0096 - accuracy: 0.3013 - mae: 0.0640
Epoch 7/100
1477/1477 [==============================] - 0s 225us/step - loss: 0.0088 - accuracy: 0.3081 - mae: 0.0606
Epoch 8/100
1477/1477 [==============================] - 0s 279us/step - loss: 0.0082 - accuracy: 0.3101 - mae: 0.0584
Epoch 9/100
1477/1477 [========================

In [6]:
Y_test.iloc[0:10, :]

,resource_allocation,jaccard_coefficient,preferential_attachment,community,sorensen_neighbours
1575,0.839080,0.919540,0.574713,0.666667,0.919540
1420,0.858289,0.834225,0.334225,0.713904,0.839572
991,0.951945,0.955378,0.411899,0.866133,0.947368
1399,0.931677,0.936335,0.338509,0.784161,0.936335
1579,0.898585,0.825472,0.365566,0.702830,0.865566
596,0.968354,0.965190,0.477848,0.778481,0.977848
1124,0.949640,0.944844,0.405276,0.856115,0.952038
1564,0.917460,0.930159,0.377778,0.860317,0.933333
1333,0.836864,0.879237,0.411017,0.760593,0.868644
1512,0.944444,0.942460,0.404762,0.857143,0.958333


In [7]:
results = regressor.predict(X_test)
results[0:20, :]

array([[0.8278412 , 0.7836548 , 0.5305677 , 0.68581736, 0.7977388 ],
       [0.8858147 , 0.89688474, 0.4035137 , 0.7989643 , 0.90170395],
       [0.96472526, 0.9732465 , 0.42007554, 0.8472571 , 0.9804982 ],
       [0.93265086, 0.93908685, 0.40861204, 0.833697  , 0.94599515],
       [0.8806856 , 0.89373666, 0.39529896, 0.81020427, 0.8975894 ],
       [0.9553513 , 0.9676637 , 0.4213266 , 0.8334745 , 0.9784336 ],
       [0.9499214 , 0.9575388 , 0.415835  , 0.84039   , 0.9653685 ],
       [0.93137866, 0.9352031 , 0.40659574, 0.8292528 , 0.943127  ],
       [0.8789358 , 0.8890287 , 0.40248874, 0.79500043, 0.8927172 ],
       [0.9472748 , 0.9555047 , 0.41355392, 0.83933216, 0.963962  ],
       [0.6345803 , 0.631066  , 0.3463575 , 0.7473288 , 0.63408494],
       [0.8572977 , 0.86595494, 0.40050998, 0.7587618 , 0.87301946],
       [0.90558356, 0.90741175, 0.40355015, 0.79698634, 0.91550416],
       [0.92041194, 0.929328  , 0.41709372, 0.8084777 , 0.9354548 ],
       [0.9149135 , 0.9208724 , 0.

In [8]:
predicted = list(map(lambda x: link_prediction_methods[x], np.argmax(results, axis=1)))
predicted[0:20]

['resource_allocation',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'community',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'sorensen_neighbours',
 'community',
 'sorensen_neighbours',
 'jaccard_coefficient',
 'sorensen_neighbours',
 'preferential_attachment']

In [9]:
import statistics

Y_score = Y_test.reset_index(drop=True)

aucs = []
for i, p in enumerate(predicted):
    aucs.append(Y_score.loc[i, p])

(statistics.mean(aucs), statistics.stdev(aucs))

(0.8656199424676011, 0.1169933765468119)

#### Baseline
- AUC mean: 0.850879
- AUC std: 0.152039

#### NN Regression
- AUC mean: 0.8667750401235926
- AUC std: 0.1124510633993589

# Best

In [17]:
best = list(map(lambda x: link_prediction_methods[x], np.argmax(Y_test.values, axis=1)))

import statistics

Y_score = Y_test.reset_index(drop=True)

aucs = []
for i, p in enumerate(best):
    aucs.append(Y_score.loc[i, p])

(statistics.mean(aucs), statistics.stdev(aucs))

(0.8845873166906277, 0.09438093765562915)

# CA

In [22]:
len(list(filter(lambda x : x, map(lambda x: best[x[0]] == x[1], enumerate(predicted)))))/

0.4121212121212121

In [23]:
len(best)

165